# LangChain: Prompt Engineering, Q&A with OpenAI and Hugging Face API 
#### [LangChain Handbook](https://github.com/pinecone-io/examples/tree/master/generation/langchain/handbook)
#### [LangChain Documentation](https://python.langchain.com/en/latest/index.html)

LangChain - a framework to build apps and pipelines around LLMs. It can be used to for chatbots, Generative Q&A (GQA), summarization, and much more. The idea is to _"chain"_ together different components => more advanced use-cases around LLMs with multiple components:
* LangChain supports __several LLM providers__, like _Hugging Face_ and _OpenAI_
* __Prompt templates__: templates for different types of prompts like "chatbot" style templates, ELI5 Q&A, etc
* __LLMs__: GPT-3, BLOOM, etc
* __Agents__: Agents perform actions using LLMs (web search, calculators, etc.) and logical loop of operations.
* __Memory__: Short-term memory, long-term memory.

[Reference](https://colab.research.google.com/github/pinecone-io/examples/blob/master/generation/langchain/handbook/00-langchain-intro.ipynb)

## Hugging Face

Hugging Face Hub API token: create an account at [HuggingFace.co](https://huggingface.co/), click on profile in the top-right corner > click *Settings* > click *Access Tokens* > click *New Token* > set *Role* to *write* > *Generate* > copy and paste the token below:

In [ ]:
import os
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

In [ ]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'HF_API_KEY'

# initialize HF LLM
flan_t5 = HuggingFaceHub(
    repo_id="google/flan-t5-xl",
    model_kwargs={"temperature":1e-10}
)

# build prompt template for simple question-answering
template = """Question: {question}

Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(
    prompt=prompt,
    llm=flan_t5
)

question = "Which NFL team won the Super Bowl in the 2010 season?"

print(llm_chain.run(question))

green bay packers


In [ ]:
# ask multiple questions
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]
res = llm_chain.generate(qs)
res

LLMResult(generations=[[Generation(text='green bay packers', generation_info=None)], [Generation(text='184', generation_info=None)], [Generation(text='john glenn', generation_info=None)], [Generation(text='one', generation_info=None)]], llm_output=None)

It is a LLM, so we can try feeding in all questions at once:

In [ ]:
# feed in all questions at once
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(
    template=multi_template,
    input_variables=["questions"]
)

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=flan_t5
)

qs_str = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
)

print(llm_chain.run(qs_str))

six


## OpenAI

In [ ]:
import os
from langchain.llms import OpenAI

In [ ]:
os.environ['OPENAI_API_KEY'] = 'OPENAI_API_KEY'
davinci = OpenAI(model_name='text-davinci-003')

We'll use the same simple question-answer prompt template as before with the Hugging Face example. The only change is that we now pass our OpenAI LLM `davinci`:

In [ ]:
# use the same Q&A prompt template as for Hugging Face
llm_chain = LLMChain(
    prompt=prompt,
    llm=davinci
)

print(llm_chain.run(question))

 The Green Bay Packers won the Super Bowl in the 2010 season.


In [ ]:
# multiple questions
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]
llm_chain.generate(qs)

LLMResult(generations=[[Generation(text=' The Green Bay Packers won the Super Bowl in the 2010 season.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' 193.04 centimeters', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' Eugene A. Cernan was the 12th person to walk on the moon. He was part of the Apollo 17 mission in December 1972.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' A blade of grass does not have any eyes.', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'total_tokens': 131, 'prompt_tokens': 75, 'completion_tokens': 56}})

In [ ]:
# this feeds in the questions in one chunk
qs = [
    "Which NFL team won the Super Bowl in the 2010 season?",
    "If I am 6 ft 4 inches, how tall am I in centimeters?",
    "Who was the 12th person on the moon?",
    "How many eyes does a blade of grass have?"
]
print(llm_chain.run(qs))


1. The New Orleans Saints 
2. 193 centimeters 
3. Harrison Schmitt 
4. Zero.


In [ ]:
# answer all question in one go
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(
    template=multi_template,
    input_variables=["questions"]
)

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=davinci
)

qs_str = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
)

print(llm_chain.run(qs_str))

The New Orleans Saints won the Super Bowl in the 2010 season.
If you are 6 ft 4 inches, you are 193.04 centimeters tall.
The 12th person on the moon was Harrison Schmitt.
A blade of grass does not have any eyes.

---